In [ ]:
"""
Konversi dataset QnA dari Excel (.xlsx) menjadi JSON.

Fitur utama:
- Membaca seluruh sheet dalam satu file Excel
- Mengambil kolom: Pertanyaan, Jawaban, dan Intent (wajib)
- Menambahkan kategori berdasarkan nama sheet
- Menangani kasus sel merge pada kolom Jawaban/Intent (forward-fill)
- Memecah sel Pertanyaan yang berisi banyak baris (newline) menjadi beberapa record
- Menghasilkan output JSON dalam bentuk list of objects
"""
# Mengimpor library bawaan untuk menulis/menyimpan data dalam format JSON
import json  
# Mengimpor Path untuk mengelola path file/folder secara portable
from pathlib import Path  
# Mengimpor tipe data untuk type hint
from typing import Any, Dict, List  
# Mengimpor pandas untuk membaca Excel dan memproses data tabular (DataFrame)
import pandas as pd  

input_path = "Dataset Augmentasi.xlsx"
output_path = "dataset_chatbot.json" 

REQUIRED_COLS = ["Pertanyaan", "Jawaban", "Intent"] 


def is_empty(value: Any) -> bool:
    """Mengecek apakah sebuah nilai dianggap kosong (None/NaN/string kosong).""" 
    # Jika nilai adalah None, maka dianggap kosong
    if value is None:
        # Mengembalikan True (kosong)  
        return True  
    # Jika nilai adalah NaN (kosong versi pandas), maka dianggap kosong
    if pd.isna(value): 
        # Mengembalikan True (kosong) 
        return True  
    # Jika nilai string tapi hanya spasi/kosong, maka dianggap kosong
    if isinstance(value, str) and value.strip() == "":  
        # Mengembalikan True (kosong)
        return True  
    # Jika tidak memenuhi kondisi di atas, berarti tidak kosong
    return False  


def normalize_text(value: Any) -> str:
    """Mengubah value menjadi string rapi (trim spasi); jika kosong kembalikan string kosong."""  
    # Memanggil is_empty untuk mengecek apakah value kosong
    if is_empty(value):  
        # Jika kosong, kembalikan string kosong
        return ""  
    # Jika tidak kosong, ubah ke string dan hapus spasi di awal/akhir
    return str(value).strip()  


def sheet_to_records(df: pd.DataFrame, category: str) -> List[Dict[str, Any]]:
    """Mengonversi satu sheet (DataFrame) menjadi list record JSON."""
    for col in REQUIRED_COLS:
        # Mengecek apakah nama kolom wajib ada di daftar kolom DataFrame
        if col not in df.columns:
            # Jika kolom tidak ditemukan, hentikan program dan tampilkan pesan error
            raise ValueError(f"Kolom '{col}' tidak ditemukan di sheet '{category}'.")

    # Melakukan forward-fill untuk menangani sel merge di Excel, di mana hanya baris pertama yang berisi nilai,
    # sedangkan baris di bawahnya terbaca NaN oleh pandas dan diisi dengan nilai dari atasnya
    df["Jawaban"] = df["Jawaban"].ffill()
    df["Intent"] = df["Intent"].ffill()

    # Menyiapkan list kosong untuk menyimpan seluruh record JSON
    records: List[Dict[str, Any]] = []

    # Melakukan iterasi untuk setiap baris pada DataFrame
    for _, row in df.iterrows():
        # Mengambil dan merapikan nilai tiap kolom
        # normalize_text akan menghapus spasi dan menangani nilai kosong
        question = normalize_text(row["Pertanyaan"])
        answer = normalize_text(row["Jawaban"])
        intent = normalize_text(row["Intent"])

        # Membentuk satu record data dalam bentuk dictionary
        record = {
            "category": category, 
            "question": question, 
            "answer": answer, 
            "intent": intent,   
        }

        # Menambahkan record ke dalam list records
        records.append(record)
    return records

def excel_to_json(excel_path: str) -> List[Dict[str, Any]]:
    """Membaca seluruh sheet Excel dan menggabungkannya menjadi satu list record JSON."""
    # Membuka file Excel untuk membaca daftar sheet
    xls = pd.ExcelFile(excel_path)
    # Membuat list kosong untuk menampung data dari semua sheet
    all_records: List[Dict[str, Any]] = []  
    
    # Iterasi semua nama sheet di file Excel
    for sheet_name in xls.sheet_names:
        # Membaca sheet tertentu ke DataFrame
        df = pd.read_excel(excel_path, sheet_name=sheet_name)
        # Mengonversi sheet menjadi record JSON
        sheet_records = sheet_to_records(df, category=sheet_name)  
        # Menggabungkan record sheet ke list utama
        all_records.extend(sheet_records) 

    return all_records


if __name__ == "__main__": 
    # Menjalankan proses konversi Excel menjadi list records JSON
    records = excel_to_json(input_path)  
    # Membuat file output jika belum ada
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)  
    # Membuka file output JSON untuk ditulis
    with open(output_path, "w", encoding="utf-8") as f:  
         # Menulis data ke JSON (rapi dengan indent=2)
        json.dump(records, f, ensure_ascii=False, indent=2) 

     # Menampilkan jumlah record yang berhasil disimpan
    print(f"Selesai! Total data valid: {len(records)}") 
     # Menampilkan lokasi file JSON output
    print(f"File JSON tersimpan di: {Path(output_path).resolve()}") 

Selesai! Total data valid: 2040
File JSON tersimpan di: C:\UNPAD MOMEN\Skripsi\Skripsiku\dataset_chatbot1.json
